# Model Selection and Regularization

In [35]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# grab the data
hitters_data = pd.read_csv('Data/Hitters.csv', index_col=0).dropna()
hitters_data.index.name = 'Player'
hitters_data.head()

,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
Player,,,,,,,,,,,,,,,,,,,
315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A


In [3]:
# one hot encode
hitter_dummies = pd.get_dummies(hitters_data[
    ['League', 'Division', 'NewLeague']
    ])
hitter_dummies.head()

,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
Player,,,,,,
315,0,1,0,1,0,1
479,1,0,0,1,1,0
496,0,1,1,0,0,1
321,0,1,1,0,0,1
594,1,0,0,1,1,0


In [4]:
# Set our response var
y = hitters_data.Salary

# Response and dummied cols
X_ = hitters_data.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')

# Combine original predictors with dummies
X = pd.concat([X_, hitter_dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X.head()

,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
Player,,,,,,,,,,,,,,,,,,
315,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0,1,1,1
479,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0,0,1,0
496,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0,1,0,1
321,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0,1,0,1
594,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,282.0,421.0,25.0,0,1,0


In [5]:
# Get our train test split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    random_state=42
                                                    )

In [9]:
len(X_)*11498/2

1511987.0

In [13]:
# Lets try this with a large alpha penalty which will make it more bias
ridge2 = Ridge(alpha=10**10)
ridge2.fit(scale(X_train), y_train)
pred = ridge2.predict(scale(X_test))
mean_squared_error(y_test, pred)

241210.9011674496